In [1]:
import tensorflow as tf
import numpy as np
import time
import codecs
from __future__ import print_function
import random

In [3]:
def read_data(file_path):
    text = codecs.open(file_path, 'r', 'utf-8' , errors='ignore').read()
    print text[:20]
    chars = list(set(text))
    text_size, vocab_size = len(text), len(chars)
    print 'text has %d characters, %d unique'%(text_size, vocab_size)
    char2id = {ch: i for i, ch in enumerate(chars)}
    id2char = { i:ch for i, ch in enumerate(chars) }
    return char2id, id2char, text

In [4]:
char2id, id2char, train_text = read_data('./input.txt')

我真的需要你
 
现在我觉得有些孤单
悲
text has 38528 characters, 1757 unique


In [5]:
vocabulary_size = len(char2id)
vocabulary_size

In [7]:
learning_rate = 1e-4
batch_size=16
num_unrollings=10

In [8]:
class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id[self._text[self._cursor[b]]]] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char[c] for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

In [9]:
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))

[u'\u6211\u771f\u7684\u9700\u8981\u4f60\n \n\u73b0\u5728', u'\n\u5feb\u4e3a\u6211\u70b9\u4eae \u70b9\u4eae \u70b9', u'\u4e2a\u65b0\u7684\u5e0c\u671b\n\u6ca1\u6709\u4e00\u4e2a\u786e', u'\n\u79bb\u5f00\u6211 \u79bb\u5f00\u6211\n\u79bb\u5f00', u'\u679c\u4f60\u611f\u5230\u4f60\u7684\u7075\u9b42\n\u53d1\u51fa', u'\u8d25\n\u6211\u4e0d\u613f\u518d\u7b49\u5f85 \u7b49\u5f85', u'\u53d1\u751f\n\u90a3\u4e9b\u873f\u8712\u7684\u5206\u660e\u662f', u' \n\u4e8c\u5341\u591a\u5c81\u90a3\u5e74\u4ed6\u4eec\u593a', u'\u5374\u6d41\u4e0d\u51fa\u773c\u6cea\n\u6211\u60f3\u558a ', u'\u6df1\u8c37\n \n\u84dd\u98ce\u7b5d\u548c\u90a3\u6a61', u'- \n\u5149\u660e\n \n\u5f53\u7070\u70ec', u'\u767d\u6865\n \n\u6211\u7684\u5144\u5f1f\u8eba\u5728', u'\u9e1f\n\u5e26\u7740\u98ce\u4e2d\u60b2\u9e23\u7684\u8349\u5e3d', u'\u5728\u7b49\u5f85\n\u7b49\u5f85\u7684\u4e00\u5207\u65e0\u6240', u'\u53ea\u662f\u7528\u4f5c\u56de\u5fc6\n\u5b83\u8ba9\u6211\u4eec', u'\u4f60\u5145\u6ee1\u4e86\u6211\u7684\u8fbd\u9614\u5fc3\u5e95\n']
[u'\u5728\u

In [86]:
text = batches2string(train_batches.next())
for i in text:
    print i

哀的自我有些辛酸
没有
红色的火光
我感到强大
期待的声音
风暴 风暴
诉你
我是无足轻重的人
意 不要介意
因为生命
胜利
即使一切都不存在
欢喜是伤悲
 
突然间
从没对那个混蛋笑过
他
意 抛弃我的所有
如果
谷鸟
带着放荡的痴心跌
当车菊草化作深秋的露水
素边喷涌
他思考着如何

 
再见 青春
再见
分裂
我的宣言也变成呢
娘 让我们在一起
无论
地
这思念就像是彻底地


In [16]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    '''
    wx = tf.Variable(tf.truncated_normal([4*vocabulary_size, num_nodes], -0.1, 0.1))
    wm = tf.Variable(tf.truncated_normal([4*num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes], trainable=False))
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    def lstm_cell(i, o, state):
        i_stacked = tf.concat(1, [i,i,i,i])
        o_stacked = tf.concat(1, [o,o,o,o])
        weights_in = tf.matmul(i_stacked, wx)
        weights_out = tf.matmul(o_stacked, wm)
        input_gate = tf.sigmoid(weights_in + weights_out + ib)
        forget_gate = tf.sigmoid(weights_in + weights_out + fb)
        update = weights_in + weights_out + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(weights_in + weights_out + ob)
        return output_gate * tf.tanh(state), state
    '''
    # Single matrix 

    ifcox = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1,0.1))
    ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
    ifcob = tf.Variable(tf.zeros([1, 4*num_nodes]))

    # Variables saving state across unrollings
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and baises.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        all_gates_state = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
        input_gate = tf.sigmoid(all_gates_state[:, 0:num_nodes])
        forget_gate = tf.sigmoid(all_gates_state[:, num_nodes:2*num_nodes])
        update = all_gates_state[:, 2*num_nodes:3*num_nodes]
        state = forget_gate*state + input_gate*tf.tanh(update)
        output_gate = tf.sigmoid(all_gates_state[:, 3*num_nodes:])
        return output_gate * tf.tanh(state), state
        
    train_data = list()
    for _ in xrange(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]
    
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)
        
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits, tf.concat(0, train_labels)))
    
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


In [14]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [18]:
num_steps = 5001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ('Initialized')
    mean_loss = 0
    for step in xrange(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in xrange(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss,
                                            train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            print ('Average loss at step', step, ':', mean_loss, 'learning rate:', lr)
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print ('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                
                print('='*80)
                for _ in xrange(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in xrange(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('='*80)
            '''
            # Measure validation set perplexity
            reset_sample_state.run()
            valid_logprob = 0
            for _ in xrange(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))
            '''

Initialized
Average loss at step 0 : 7.47007036209 learning rate: 10.0
Minibatch perplexity: 1754.73
村曲崖意眼铺扉场夏倔芒树恣妙虫奄转着罪充悉股茶唤闯谬求满绽抵h至件葬赢唇咒工阑控垮以惊雾读兵裙臂滴我一霜免库髦议唱握歌欲死杨铃酸友专拜阔怅尊凭星娇封砍懵病童响潦
怜春暧抖睡的树给缓策际赌表烈否0未杯万偿绝颗汤流蜿短朝抱魔美敬声度紧社祖异问未措拔赦挡加谈垃打莎湾格萎扑兜狂原块仔梦押坚私晴呼背并李十喀锋站煤弦银执脆鹰教途欢铭
课而祖拆室默缩黄缘坦别器谎游跄微冒忧惶谜杀晕城醒否工砸访芳啤识众辰用及厂烟罪挽瑰获端恳干次惑牢知丐麦奈啸必零危珊灭珊都佛边阳努挽华项还闻忐卑约的怒雀简袋尖肤涌聊
无陋内呦衰戒纹告推街弦摸宴程低寒背验朝纷弹枭挫昂世4鳞随饿抓敬侣谬甚法田任颗别蓄挤觉人虹策璨纷轰山伪了睡夏断各红滑胶乌伸乎必心获贝备币被抵温屈否袭肉岸暧脚让彼儿
觅公拒圣巧卖糟盘边率诞议驱声粪伙最麦听百程倾河何掩自睁扬伤苍屁隐娘悠命初喧题具春话占挺自有限拥掬占愤外拍6星弟辉演熟痛长弦抹自官私唤纷啊嘴沮引治谅贷迪润马于肯敲
Average loss at step 100 : 5.85017663479 learning rate: 10.0
Minibatch perplexity: 238.24
Average loss at step 200 : 5.3165617609 learning rate: 10.0
Minibatch perplexity: 154.55
Average loss at step 300 : 5.0546865654 learning rate: 10.0
Minibatch perplexity: 146.24
Average loss at step 400 : 4.82865390778 learning rate: 10.0
Minibatch perplexity: 82.79
Average loss at step 500 : 4.66981710911 learning rate: 10.0
Minibatch perplexity: 104.76
Average loss at step 600 : 4.46629605293 learni